In [111]:
#imports:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ###
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
#from sklearn import metrics
#import matplotlib.pyplot as plt

In [112]:
data_ANN = pd.read_csv('GoldWeekly2001-2021.csv')

data_ANN = data_ANN.drop('Did_Win', axis=1)
data_ANN = data_ANN.drop('Volume', axis=1)

for i in range(len(data_ANN)):
    if data_ANN.iloc[i,-1] == 0:
        continue
    else:
        data_ANN.iloc[i,-1] = data_ANN.iloc[i,-1] / data_ANN.iloc[i,-2] * 100

data_ANN.head(n=4)

,Date,Open,High,Low,Close,Profit
0,1/1/2001,268.399994,268.399994,267.299988,268.000000,-1.529853
1,1/8/2001,268.000000,268.000000,263.899994,263.899994,0.151570
2,1/15/2001,263.299988,264.299988,263.200012,264.299988,-0.567537
3,1/22/2001,266.399994,266.399994,262.799988,262.799988,1.636232


In [113]:
from sklearn import preprocessing

Label = preprocessing.LabelEncoder()

infoFromLabel = Label.fit_transform(data_ANN['Open'])
#infoFromLabel

In [114]:
#Preprocessing data:
data_labels = data_ANN['Date'].copy()
data_ANN = data_ANN.drop('Date', axis = 1)

#Split data for training and testing
X_train, X_Test_Val = train_test_split(data_ANN, test_size = 0.4, train_size = 0.6, shuffle = False)
X_test, X_val = train_test_split(X_Test_Val, test_size = 0.5, train_size = 0.5, shuffle = False)
#Removing the labels from the data.
train_labels = X_train['Profit'].copy()
X_train = X_train.drop('Profit', axis = 1)

test_labels = X_test['Profit'].copy()
X_test = X_test.drop('Profit', axis = 1)

val_labels = X_val['Profit'].copy()
X_val = X_val.drop('Profit', axis = 1)


In [115]:
#Multi-layer-perceptron
MLP = MLPRegressor(hidden_layer_sizes=(5,5,5), max_iter=20_000)
MLP.fit(X_train, train_labels)

results = MLP.predict(X_test)

result_df = pd.DataFrame(X_test)
result_df = result_df.assign(Profit = test_labels, Predictions = results)

display(result_df)

,Open,High,Low,Close,Profit,Predictions
626,1655.400024,1693.800049,1625.699951,1648.099976,0.722045,0.039352
627,1656.500000,1677.300049,1643.800049,1660.000000,1.602408,0.255763
628,1665.400024,1694.599976,1664.099976,1686.599976,-1.790582,0.713972
629,1690.300049,1694.699951,1656.400024,1656.400024,0.784835,-0.680328
630,1651.599976,1683.000000,1651.599976,1669.400024,-0.203667,0.689871
...,...,...,...,...,...,...
830,1188.000000,1194.800049,1160.000000,1175.099976,-1.336052,-0.204164
831,1181.800049,1186.699951,1156.000000,1159.400024,-2.078659,-0.347941
832,1158.300049,1164.000000,1123.900024,1135.300049,-0.299483,-0.434868
833,1139.400024,1140.699951,1126.500000,1131.900024,1.599079,-0.000089
